# Generating Datasets

In [1]:
import cv2 as cv
import os
import numpy as np
from PIL import Image

In [2]:
if not os.path.exists("faces"):
    os.mkdir("faces")
cap = cv.VideoCapture(0)

face_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')
face_id = input('Enter your Id: ')

count = 0

while True:
    ret, frame = cap.read()
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.5, 5)

    for (x,y,w,h) in faces:
        cv.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)

        count += 1
        face_image = gray[y:y+h, x:x+w]
        cv.imwrite(f'faces/{face_id}_{count}.jpg', face_image)

    
    cv.putText(frame, str(count), (frame.shape[1] // 2, frame.shape[0] - 50), cv.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
    cv.imshow('Captured Image', frame)
        
    if count >= 400:
        break

    if cv.waitKey(1) &0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()
cv.waitKey(1)

Enter your Id:  4


2025-02-19 15:26:24.279 Python[6282:313030] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-19 15:26:24.279 Python[6282:313030] +[IMKInputSession subclass]: chose IMKInputSession_Modern


-1

# Training Datasets

In [7]:
def data_classifier(faces_dir):
    # Get all image paths from the faces_dir
    path = [os.path.join(faces_dir, f) for f in os.listdir(faces_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]

    face = []
    ids = []

    for image in path:
        img = cv.imread(image, cv.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Failed to load image: {image}")
            continue
        
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[-1].split('_')[0])

        face.append(imageNp)
        ids.append(id)

    ids = np.array(ids)

    # Training classifier
    clf = cv.face.LBPHFaceRecognizer_create()
    clf.train(face, ids)
    clf.write("trained_classifier.xml")

    print("Training complete. Classifier saved as 'trained_classifier.xml'.")

# Call the function
data_classifier("faces")


Training complete. Classifier saved as 'trained_classifier.xml'.


# Face Recognizer
